In [1]:
n = 10^3

nu_ = rand((1, 2, 3), n)

;

Generators do not alloacte;

In [2]:
@btime count(nu == 1 for nu in $nu_);
@btime count((nu == 1 for nu in $nu_));

  133.655 ns (0 allocations: 0 bytes)
  133.952 ns (0 allocations: 0 bytes)


Making a vector object results in an unnecessary allocation;

In [3]:
@btime count([nu == 1 for nu in $nu_]);

  283.424 ns (1 allocation: 1.06 KiB)


Mapping a function allocates.
But letting `count` chain this mapping speeds up the computation;

In [4]:
@btime count(map(==(1), $nu_));

@btime count(==(1), $nu_);
@btime count(nu -> nu==1, $nu_);

  380.283 ns (1 allocation: 1.06 KiB)
  134.064 ns (0 allocations: 0 bytes)
  134.064 ns (0 allocations: 0 bytes)


Vectorization is usually fast;

In [5]:
@btime $nu_ .* 1 ./ 2 .+ 3 .- 4 .^ 5;

@btime [nu * 1 / 2 + 3 - 4 ^ 5 for nu in $nu_];

  223.769 ns (1 allocation: 7.94 KiB)
  237.553 ns (1 allocation: 7.94 KiB)


But vectorization is slow here because it makes a `BitVector` instead of `Vector{Bool}`, which is denser and faster;

In [6]:
@btime count($nu_ .== 1);

  415.201 ns (3 allocations: 4.41 KiB)


In [7]:
supertypes(typeof([nu == 1 for nu in nu_]))

(Vector{Bool}, DenseVector{Bool}, AbstractVector{Bool}, Any)

In [8]:
supertypes(typeof(nu_ .== 1))

(BitVector, AbstractVector{Bool}, Any)

Generators are usually the best.

Unless there is a specific function, which is `count(function, container)` here.

Check vectorization intermediate types.